In [6]:
pip install unstructured

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     -------------------- ----------------- 524.3/981.5 kB 1.4 MB/s eta 0:00:01
     ------------------------------ ------- 786.4/981.5 kB 1.5 MB/s eta 0:00:01
     ------------------------------------ 981.5/981.5 kB 990.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.8 MB 1.7 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.8 MB 1.9 MB/s eta 0:00:01
   ----

  DEPRECATION: Building 'langdetect' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'langdetect'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  You can safely remove it manually.


In [9]:
pip install unstructured python-pptx



   ---------------------------------------- 0/2 [XlsxWriter]
   ---------------------------------------- 0/2 [XlsxWriter]
   -------------------- ------------------- 1/2 [python-pptx]
   -------------------- ------------------- 1/2 [python-pptx]
   -------------------- ------------------- 1/2 [python-pptx]
   -------------------- ------------------- 1/2 [python-pptx]
   ---------------------------------------- 2/2 [python-pptx]

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gemma:2b")

model

ChatOllama(model='gemma:2b')

In [11]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPowerPointLoader

loader = DirectoryLoader(
    path=r"..\data\books",
    glob="**/*.pptx",
    loader_cls=UnstructuredPowerPointLoader
)

docs = loader.load()


In [12]:
data = loader.load()
data

[Document(metadata={'source': '..\\data\\books\\Feature Engineering.pptx'}, page_content="Feature Engineering\n\nFeature Engineering is like giving your data a makeover before it meets a machine learning model.\n\nIt means cleaning, creating, transforming, selecting, or removing features (columns) so the model can understand patterns more clearly.\n\nIn short — it’s the art of turning raw data into smart data that helps machines learn better.\n\nWhy do we need feature engineering?\n\nWe need feature engineering because it helps improve model performance by turning raw data into meaningful inputs — making patterns clearer and predictions more accurate.\n\n10\n\n\n\nImagine you start building a machine learning model using raw data like this:\n\nIf you train a model directly on this, it’ll struggle — because of missing ages, inconsistent formats, and unclear values.\n\nThat’s where feature engineering comes in. It cleans and prepares such data:\n\nFills missing values, ﬁxes date formats\

In [13]:
type(data)

list

In [14]:
len(data)

2

In [15]:
doc_1 = data[0]
doc_2 = data[1]

In [27]:
from langchain_core.prompts import PromptTemplate

# Prompt
prompt_1 = PromptTemplate(
    template="""
    Tell me the core theme of the content below
    and explain how it is useful.

    Content:
    {text}
    """,
    input_variables=["text"]
)

prompt_1

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n    Tell me the core theme of the content below\n    and explain how it is useful.\n\n    Content:\n    {text}\n    ')

In [28]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser

StrOutputParser()

In [32]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

chain = RunnableParallel({
    "summary_1": RunnableLambda(lambda x: {"text": x["doc_1"]}) | prompt_1 | model | parser,
    "summary_2": RunnableLambda(lambda x: {"text": x["doc_2"]}) | prompt_1 | model | parser,
})

chain

{
  summary_1: RunnableLambda(...)
             | PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n    Tell me the core theme of the content below\n    and explain how it is useful.\n\n    Content:\n    {text}\n    ')
             | ChatOllama(model='gemma:2b')
             | StrOutputParser(),
  summary_2: RunnableLambda(...)
             | PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n    Tell me the core theme of the content below\n    and explain how it is useful.\n\n    Content:\n    {text}\n    ')
             | ChatOllama(model='gemma:2b')
             | StrOutputParser()
}

so, x will be a dictionary of two passed pairs, in lambda func, we fetch the required data from x-dictionary.

In [33]:
response = chain.invoke({
    "doc_1": doc_1.page_content,
    "doc_2": doc_2.page_content,
})

print(response)


{'summary_1': 'Sure, here is the core theme of the content and its usefulness:\n\n**Core Theme: Feature Engineering is a crucial step in building accurate machine learning models.**\n\n**Useful Aspects:**\n\n- Feature engineering helps address the issue of data noise and missing values.\n- It prepares data for the machine learning model by cleaning, encoding categorical and numerical data, and transforming features.\n- Feature engineering can significantly improve model performance by improving the quality and consistency of data.\n- It ensures that the model is trained on data that is relevant to the real-world application.\n- By focusing on feature engineering, you can create features that capture important patterns and relationships in the data, leading to better model performance.\n\nOverall, feature engineering is an essential step in building robust and accurate machine learning models by cleaning, transforming, and creating features that effectively capture the underlying patter

In [35]:
response['summary_1']

'Sure, here is the core theme of the content and its usefulness:\n\n**Core Theme: Feature Engineering is a crucial step in building accurate machine learning models.**\n\n**Useful Aspects:**\n\n- Feature engineering helps address the issue of data noise and missing values.\n- It prepares data for the machine learning model by cleaning, encoding categorical and numerical data, and transforming features.\n- Feature engineering can significantly improve model performance by improving the quality and consistency of data.\n- It ensures that the model is trained on data that is relevant to the real-world application.\n- By focusing on feature engineering, you can create features that capture important patterns and relationships in the data, leading to better model performance.\n\nOverall, feature engineering is an essential step in building robust and accurate machine learning models by cleaning, transforming, and creating features that effectively capture the underlying patterns and relatio

In [36]:
response['summary_2']

'**Core Theme**: Machine learning (ML) is a branch of artificial intelligence (AI) that allows computers to learn from data and make predictions or decisions without being explicitly programmed.\n\n**How it is useful**:\n\n* Solves difficult problems: ML can find patterns and answers in situations that are too complex for humans to solve easily.\n* Handles huge amounts of data: ML can study and learn from millions of data points much faster than people can.\n* Automates repetitive tasks: ML can do the same kind of work over and over again without getting tired or making mistakes.\n* Improves over time: The more data it sees, the smarter it becomes and the better its predictions get.'

In [37]:
chain.get_graph().print_ascii()

     +------------------------------------+      
     | Parallel<summary_1,summary_2>Input |      
     +------------------------------------+      
                ***           ***                
              **                 **              
            **                     **            
    +--------+                    +--------+     
    | Lambda |                    | Lambda |     
    +--------+                    +--------+     
          *                           *          
          *                           *          
          *                           *          
+----------------+            +----------------+ 
| PromptTemplate |            | PromptTemplate | 
+----------------+            +----------------+ 
          *                           *          
          *                           *          
          *                           *          
  +------------+                +------------+   
  | ChatOllama |                | ChatOllama |   
